# Using [`depfinder`](https://github.com/ericdill/depfinder) to discover the `deathbeds` dependencies.



In [1]:
    from pathlib import Path
    from IPython.utils.capture import capture_output
    from IPython import get_ipython
    from yaml import safe_load
    from collections import defaultdict
    import deathbeds
    import depfinder

* Loop all the notebooks in the deathbeds project.

In [ ]:
    if __name__ == '__main__':
        deps = defaultdict(list)
        for path in map(str, Path(deathbeds.__file__).parent.glob('*.ipynb')): 
            with capture_output() as out:
                !depfinder --yaml --conda $path
            if not out.stdout.strip().startswith('Traceback'):
                current = safe_load(out.stdout.strip())
                if current:
                    for key in current: 
                        deps[key] += current[key]

        requirements = set(
            sum((list(object for object in value if not object.startswith('_')) for key, value in deps.items() 
                 if key not in ['builtin']), []))

        if 'disqus' in requirements: requirements.remove('disqus')
        if 'deathbeds' in requirements: requirements.remove('deathbeds')
        if 'opencv' in requirements: 
            requirements.remove('opencv')
            requirements.update({'opencv-python'})

        list(map(requirements.add, {
            'poser', 'importnb', 'depfinder', 'pytest', 'jupyter', 'graphviz'
        }));


        Path('../requirements.txt').write_text("\n".join(requirements))